# Analyse des symétries de la maille conventionnelle du CaCN2
## Chargement des modules et des fichiers

In [1]:
import numpy as np
import pandas as pd
from mp_api.client import MPRester
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from jupyter_jsmol.pymatgen import quick_view
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
mp_key = "wjMeCvxez5rqgsz8tYQIQIC7njLkvYiq"
mp_id = "mp-4124"

In [3]:
print("Téléchargement du fichier")
with MPRester(mp_key) as m:
    prim_struc = m.get_structure_by_material_id(mp_id)
print("Téléchargement effectué avec succès")

conv_struc = SpacegroupAnalyzer(prim_struc).get_conventional_standard_structure()
analyze_struc = SpacegroupAnalyzer(conv_struc)


Téléchargement du fichier


Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Téléchargement effectué avec succès


In [16]:
view = quick_view(conv_struc, "packed", conventional = True)
display(view)
view.script('select calcium; color lightsalmon')
view.script('select carbon; color gold')
view.script('select nitrogen; color palegreen')

JsmolView(layout=Layout(align_self='stretch', height='400px'))

## Récupérationd es positions des atomes et des symétries

Le CaCN2 appartient au groupe R$\bar{3}$m, qui est le groupe numéro 166. Analysons 3 symétries sur 3 atomes différents qui font partie de la maille conventionnelle.

In [5]:
#Sites des atomes dans la maille cristalline 
sites = conv_struc.sites
for i in range(len(sites)):  # Parcours de tous les sites
    print(f"Atome {i}: {sites[i].specie}, \n Coordonnées fractionnaires: {sites[i].frac_coords}, \n Coordonnées cartésiennes :{conv_struc.lattice.get_cartesian_coords(sites[i].frac_coords)}")
    
#On choisit les coordonnées de 1Ca, 1C et 1N
Ca_pos = sites[1].frac_coords
C_pos = sites[3].frac_coords
N_pos = sites[11].frac_coords

#Opérations de symétries ponctuelles
symmops = SpacegroupAnalyzer(conv_struc).get_point_group_operations()


Atome 0: Ca, 
 Coordonnées fractionnaires: [0. 0. 0.], 
 Coordonnées cartésiennes :[0. 0. 0.]
Atome 1: Ca, 
 Coordonnées fractionnaires: [0.66666667 0.33333333 0.33333333], 
 Coordonnées cartésiennes :[ 1.84652782 -1.06609334  4.9387747 ]
Atome 2: Ca, 
 Coordonnées fractionnaires: [0.33333333 0.66666667 0.66666667], 
 Coordonnées cartésiennes :[1.84652782 1.06609334 9.87754941]
Atome 3: C, 
 Coordonnées fractionnaires: [0.33333333 0.66666667 0.16666667], 
 Coordonnées cartésiennes :[1.84652782 1.06609334 2.46938735]
Atome 4: C, 
 Coordonnées fractionnaires: [1.  1.  0.5], 
 Coordonnées cartésiennes :[ 3.69305565e+00 -4.44089210e-16  7.40816206e+00]
Atome 5: C, 
 Coordonnées fractionnaires: [0.66666667 0.33333333 0.83333333], 
 Coordonnées cartésiennes :[ 1.84652782 -1.06609334 12.34693676]
Atome 6: N, 
 Coordonnées fractionnaires: [0.33333333 0.66666667 0.08363866], 
 Coordonnées cartésiennes :[1.84652782 1.06609334 1.23921745]
Atome 7: N, 
 Coordonnées fractionnaires: [0.33333333 0.66

# Première symétrie
J'ai choisi d'appliquer une inversion $\bar{1}$. Il s'agit d'une inversion centrale. Cette opération transforme des coordonnées (x, y, z) en (-x, -y, -z). La matrice de cette opération est  \begin{pmatrix} -1& 0&0 \\
0&-1&0 \\ 0&0&-1 
\end{pmatrix}
Appliquons cette opération sur le Ca que nous avons choisi

In [6]:
i_symmop1 = 2
sym1 = symmops[i_symmop1-1]

#Matrice de rotation et vecteur tau
print("Matrice de transformation et vecteur tau")
print(sym1)

#Coordonnées du Ca
print("Coordonnées initiales d'un atome Ca")
print(Ca_pos)

#Coordonnées après l'opération
print("Coordonnées après avoir appliqué la symétrie")
Op_Ca = sym1.operate(Ca_pos)
print(Op_Ca)

Matrice de transformation et vecteur tau
Rot:
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.]
Coordonnées initiales d'un atome Ca
[0.66666667 0.33333333 0.33333333]
Coordonnées après avoir appliqué la symétrie
[-0.66666667 -0.33333333 -0.33333333]


On remarque qu'on arrive bien à (-x, -y, -z). Si on remet les coordonnées obtenues dans la maille (qui va de 0 à 1 pour les coordonnées en x y et z), on tombe sur (0.333  0.667  0.667), ce qui correspond aux coordonnées de l'atome 3, qui est un atome de Ca. Cela nous permet de vérifier la symétrie. 

In [7]:
#Affichage de la symétrie
view1 = quick_view(prim_struc, "packed", conventional = True)
display(view1)
view1.script('select calcium; color lightsalmon')
view1.script('select carbon; color gold')
view1.script('select nitrogen; color palegreen')
view1.script('draw SYMOP ' + str(i_symmop1) + ' {atomno = ' + str(6)+ '}')

JsmolView(layout=Layout(align_self='stretch', height='400px'))

Grâce à JsMol, nous pouvons visualiser l'inversion autour du centre de symétrie, qui est l'atome de Ca à la position (0,0,0)

# Deuxième symétrie 
J'ai choisi d'appliquer une rotation d'ordre 3 autour de l'axe z $[0,0,1]$. Cette opération transforme des coordonnées (x, y, z) en (-y, x-y, z). La matrice de cette transformation est \begin{pmatrix} 0& -1&0 \\1&-1&0 \\ 0&0&1 
\end{pmatrix}. 
Appliquons cette transformation à l'atome C sélectionné. 

In [8]:
i_symmop2 =3
sym2 = symmops[i_symmop2-1]

#Matrice de rotation et vecteur tau
print("Matrice de transformation et vecteur tau")
print(sym2)

#Coordonnées du C
print("Coordonnées initiales d'un atome C")
print(C_pos)

#Coordonnées après l'opération
print("Coordonnées après avoir appliqué la symétrie")
Op_C = sym2.operate(C_pos)
print(Op_C)

Matrice de transformation et vecteur tau
Rot:
[[ 0. -1.  0.]
 [ 1. -1.  0.]
 [ 0.  0.  1.]]
tau
[0. 0. 0.]
Coordonnées initiales d'un atome C
[0.33333333 0.66666667 0.16666667]
Coordonnées après avoir appliqué la symétrie
[-0.66666667 -0.33333333  0.16666667]


On arrive bien aux coordonnées (-y, x-y, z). Si on cherche dans les sites occupés par les atomes, on constate qu'on tombe bien sur un atome le même atome de C. 

In [9]:
#Affichage de la symétrie
view2 = quick_view(prim_struc, "packed", conventional = True)
display(view2)
view2.script('select calcium; color lightsalmon')
view2.script('select carbon; color gold')
view2.script('select nitrogen; color palegreen')
view2.script('draw SYMOP ' + str(i_symmop2) + ' {atomno = ' + str(2) + '}')

JsmolView(layout=Layout(align_self='stretch', height='400px'))

# Troisième symétrie 
Pour terminer, j'ai choisi d'appliquer une roto-inversiond'ordre 3 ($\bar{3}$). Nous allons l'appliquer à l'atome N sélectionné ci-dessus. La matrice de cette transformation est \begin{pmatrix} 0& 1&0 \\-1&1&0 \\ 0&0&-1 
\end{pmatrix}. On a une transformation qui correspond à une rotation de 120° suivie d'une réflexion. Cette rotoinversion est parallèle à la direction $[0 0 1]$. On envoie donc les coordonnées (x, y, z) vers (y, -x+y, -z). 

In [10]:
#Affichage de la symétrie
i_symmop3 =4
sym3 = symmops[i_symmop3-1]

#Matrice de rotation et vecteur tau
print("Matrice de transformation et vecteur tau")
print(sym3)

#Coordonnées du C
print("Coordonnées initiales d'un atome N")
print(N_pos)

#Coordonnées après l'opération
print("Coordonnées après avoir appliqué la symétrie")
Op_N = sym3.operate(N_pos)
print(Op_N)

Matrice de transformation et vecteur tau
Rot:
[[ 0.  1.  0.]
 [-1.  1.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.]
Coordonnées initiales d'un atome N
[0.66666667 0.33333333 0.91636134]
Coordonnées après avoir appliqué la symétrie
[ 0.33333333 -0.33333333 -0.91636134]


On arrive aux coordonnées $(0.33, 0.67, 0.08)$ qui correspondent également aux coordonnées d'un atome de N dans la maille conventionnelle. 

In [13]:
#Affichage de la symétrie
view3 = quick_view(prim_struc, "packed", conventional = True)
display(view3)
view3.script('select calcium; color lightsalmon')
view3.script('select carbon; color gold')
view3.script('select nitrogen; color palegreen')
view3.script('draw SYMOP ' + str(i_symmop3) + ' {atomno = ' + str(9) + '}')

JsmolView(layout=Layout(align_self='stretch', height='400px'))